In [1]:
import warnings
warnings.filterwarnings('ignore')

from scripts import utils

import os
from pprint import pprint 
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [1]:
from trulens_eval import Tru

tru = Tru(database_url='sqlite:///db/rag_triad.sqlite') 

🦑 Tru initialized with db url sqlite:///db/rag_triad.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents])) 

In [5]:
from scripts.utils import build_sentence_window_index
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
) 

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
from scripts.utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [7]:
output = sentence_window_engine.query("How do you create your AI portfolio?")

pprint(output.response)

('To create your AI portfolio, you should focus on building a collection of '
 'projects that demonstrate your skill progression. This can be achieved by '
 'starting with simpler projects and gradually increasing the complexity as '
 'you gain more experience. Additionally, it is important to ensure that your '
 'portfolio showcases a variety of AI techniques and technologies to highlight '
 'your versatility. By following this approach, you can effectively showcase '
 'your abilities and attract potential employers in the field of AI.')


### Feedback Functions

In [8]:
import nest_asyncio

nest_asyncio.apply()

In [9]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI() 

#### Answer Relevance

In [10]:
from trulens_eval import Feedback

# cot - chain of thought 

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input().on_output()

# .on_input_output() 

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


#### Context Relevance

In [11]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [12]:
import numpy as np

# qs - question statement 
f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons, name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


#### Groundedness

In [13]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [14]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


### Evaluation of the RAG application

In [15]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[f_qa_relevance, f_qs_relevance, f_groundedness],
)

In [16]:
eval_questions = []
with open('texts/eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)

In [17]:
eval_questions.extend(["How can I be successful in AI?", "Who is Bola Ahmed Tinubu?"])

In [18]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?',
 'Who is Bola Ahmed Tinubu?']


In [19]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [20]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_94e1ca95b08637234dbdbc7b2bf67677,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_94e1ca95b08637234db...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T21:34:59.902727"", ""...",2024-01-15T21:35:09.976337,0.8,1.000,0.40,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'question': 'What are the keys to b...,10,514,0.000788
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_77ac76311b14bcdd4fe88431e017deef,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_77ac76311b14bcdd4fe...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T21:35:11.340892"", ""...",2024-01-15T21:35:24.625583,0.9,0.950,0.65,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,13,662,0.001045
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_098d3b98c3d79d7a80bd1169de1714ac,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_098d3b98c3d79d7a80b...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T21:35:25.814916"", ""...",2024-01-15T21:35:43.219634,0.9,0.375,0.20,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,17,544,0.000871
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_55a4f5ea16367199084aa8633d3eeb20,"""What are some good habits to develop for a su...","""Developing good habits in eating, exercise, s...",-,"{""record_id"": ""record_hash_55a4f5ea16367199084...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T21:35:44.499558"", ""...",2024-01-15T21:35:54.746675,0.9,0.500,0.50,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,10,447,0.000696
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3f1872d5939d1a1e015e7cc277544714,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_3f1872d5939d1a1e015...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-15T21:35:56.051698"", ""...",2024-01-15T21:36:07.801922,0.9,0.500,0.45,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,11,547,0.000864


In [21]:
feedback

['Context Relevance', 'Answer Relevance', 'Groundedness']

In [22]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.""",0.40,0.8,1.000
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively with a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.""",0.65,0.9,0.950
2,"""What is the importance of networking in AI?""","""Networking is important in AI because it allows individuals to connect with others who have experience and knowledge in the field. By reaching out to people in their network, such as friends or alumni, individuals can receive guidance and support as they navigate their AI career. Additionally, attending meetups and engaging in informational interviews can help individuals expand their network and learn from professionals who hold positions aligned with their interests. This networking can provide valuable insights, help individuals stay updated on the evolving field of AI, and potentially open up opportunities for collaboration or job prospects in the future.""",0.20,0.9,0.375
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in eating, exercise, sleep, personal relationships, work, learning, and self-care can be beneficial for a successful career. These habits can help individuals move forward while staying healthy and contribute to their overall success in their professional lives.""",0.50,0.9,0.500
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by helping others even as one focuses on their own career growth. By aiming to lift others during every step of their own journey, individuals can achieve better outcomes for themselves. This can create a positive reputation and network, which can lead to new opportunities and collaborations. Additionally, helping others can provide a sense of fulfillment and purpose, which can contribute to overall career satisfaction and well-being.""",0.45,0.9,0.500
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological phenomenon where individuals doubt their own abilities and fear being exposed as a fraud, despite evidence of their competence. In the context of AI, newcomers to the field sometimes experience imposter syndrome, questioning whether they truly belong in the AI community and if they are capable of succeeding in the field. It is important to address imposter syndrome in order to encourage individuals to continue growing in AI and not let self-doubt hinder their progress.""",0.85,0.9,1.000
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",0.65,0.8,1.000
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",0.55,0.9,1.000
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating with stakeholders 

In [23]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
App_1,0.5,0.9,0.752917,10.666667,0.000865


In [24]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.161:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.stop_dashboard()